In [13]:
import requests, pprint, http.client, json

In [29]:
#   movies.json

conn = http.client.HTTPSConnection("api.themoviedb.org")

payload = "{}"
dataObject = []
for i in range(20):
   conn.request("GET", "/3/movie/popular?page="+f"{i+1}"+"&language=ko-kr&api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
   res = conn.getresponse()
   data = res.read()
   dataObject += json.loads(data)['results']

with open('movies1.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [30]:
movie_id = []
for i in range(len(dataObject)):
    movie_id.append(dataObject[i]["id"])

In [ ]:
#  actor1.json

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
raw = []
dataObject = []

for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    dataObject += [{"movie_id" : i}]
    
    for j in range(3):
        try:
            raw += data['cast']
            dataObject.append(raw[j])
        except:
            pass
        raw = []
    with open('actor1.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [ ]:
conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
raw = []
dataObject = []
dataDetail = []

for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    dataObject += [{"movie_id" : i}]
    
    for j in range(3):
        try:
            raw += data['cast']
            dataObject.append(raw[j])
        except:
            pass
        raw = []
    dataDetail += [dataObject]
    with open('actor2.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [31]:
#  actor.json 완성작

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
raw = []
dataObject = []

for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
#     conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    for j in range(min(3, len(data['cast']))):
        dataDetail = {}
        dataDetail['pk'] = data['cast'][j]['id']
        dataDetail['model'] = 'movies.actor'
        dataDetail['fields'] = {
            'name': data['cast'][j]['name'],
            'img': data['cast'][j]['profile_path'],
            'acted': i
        }
        dataObject.append(dataDetail)

    with open('actor.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

KeyError: 'cast'

In [28]:
#  director.json 완성작

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
dataObject = []


for i in movie_id:
    conn.request("GET", "/3/movie/"+f"{i}"+"/credits?api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
    res = conn.getresponse()
    data = json.loads(res.read())
    for j in range(len(data['crew'])):
        if data['crew'][j]['job'] == 'Director':
            dataDetail = {}
            dataDetail['pk'] = data['crew'][j]['id']
            dataDetail['model'] = 'movies.director'
            dataDetail['fields'] = {
             'name': data['crew'][j]['name'],
            'job': data['crew'][j]['job'],
            'img': data['crew'][j]['profile_path'],
            'directed': i
            }
            dataObject.append(dataDetail)
            
    with open('director.json', 'w', encoding='utf-8') as f:
        json.dump(dataObject, f, ensure_ascii=False, indent=4)

In [ ]:
#  genres.json 완성작

conn = http.client.HTTPSConnection("api.themoviedb.org")
payload = "{}"
dataObject = []
conn.request("GET", "/3/genre/movie/list?language=ko-kr&api_key=f55decdd8b4533ff751d5e5df47ac372", payload)
res = conn.getresponse()
data = res.read()
dataObject += json.loads(data)['genres']

with open('genres.json', 'w', encoding='utf-8') as f:
    json.dump(dataObject, f, ensure_ascii=False, indent=4)